# Assignment 2
Group 14	Nintendo shop - https://store.nintendo.nl/nl  
Zijian Dong -i6226172	Anqi Fan -i6272224	Qi Cui -i6288444	Tianyu Wei -i6231911	Zhefan Cheng -i6267501

## Graph generating:

In [1]:
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, SDO, RDFS
from rdflib import Graph


In [2]:
# Make a graph
g = Graph()
NS = Namespace('https://store.nintendo.nl/nl/')
#--------------Categories------------------------------------

console = NS['consoles']
games = NS['games']
merchandise = NS['merchandise'] #周边merchandise
franchise = NS['franchises'] #独家ip 和其它的会重复Exclusive IP and others will be duplicated
new_product = NS['new']

#--------Sub-Categories--------------------------------------
#--------Sub Categories for consoles-------------------------
switch = NS['nintendo-switch-consoles'] #是console的product
switch_bundle = NS['nintendo-switch-bundles'] #是console的product
accessory = NS['accessories'] #是switch的配件
#-------Sub Categories for merchandise-----------------
apparel = NS['apparel']
home_and_gifts = NS['home-and-gifts']


In [3]:
#--------Instance--------------------------------------

#Define types for Mainpage
g.add((console, RDF.type, SDO.ProductGroup)) # triple set："_SubjectType", "_PredicateType", "_ObjectType"
g.add((games, RDF.type, SDO.ProductGroup))
g.add((merchandise, RDF.type, SDO.ProductGroup))
g.add((franchise, RDF.type, SDO.ProductGroup))
g.add((new_product, RDF.type, SDO.ProductGroup))

#Define subclasses of main categories
g.add((switch, RDFS.subClassOf, console))
g.add((switch_bundle, RDFS.subClassOf, console))
g.add((accessory, RDFS.subClassOf, console))

g.add((apparel, RDFS.subClassOf, merchandise))
g.add((home_and_gifts, RDFS.subClassOf, merchandise))


<Graph identifier=Nfd9b6956f8084b5790cb06a8ec9ac8e8 (<class 'rdflib.graph.Graph'>)>

In [4]:
import pandas as pd

df = pd.read_csv("group-14_ken3140_webshop.csv")

for i,row in df.iterrows():
    # Check if URI is empty, skip the empty ones
    if not pd.isna(row["Item URI"]) :
        item = NS[row["Item URL"].replace("https://store.nintendo.nl/nl/","").replace(" ","")]
        for category in row['Category'].split(","):
            g.add((item, SDO.item, locals()[category]))
        
        g.add((item, RDF.type, SDO.Product))
        g.add((item, SDO.name, Literal(row["Value 1"])))
        g.add((item, SDO.description, Literal(row["Value 2"])))
        g.add((item, SDO.sku, Literal(row["Value 3"])))
        g.add((item, SDO.price, Literal(row["Value 4"])))
        g.add((item, SDO.brand, Literal("Nintendo"))) 
    #    graph.add((product, schema[row["link"]],eval(row["type"])))


In [5]:
g.bind('NS', NS)

print(g.serialize('group-14_ken3140_webshop.ttl',format='turtle'))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


## SPARQL:

In [6]:
from rdflib import Graph

graph = Graph()
graph.parse("group-14_ken3140_webshop.ttl", format="turtle")

<Graph identifier=Ne1f51b298e984bb4be71504424ed2206 (<class 'rdflib.graph.Graph'>)>

In [7]:
'''
AN EXAMPLE
'''
item_info_query = """
SELECT ?price WHERE {
  ?s schema:name 'Splatoon 3'.
  ?s schema:price ?price.
}
"""

results = graph.query(item_info_query)
for row in results:
    print(row.price)

59.99


### Questions:

A. For a given item (select a random item from your RDF Graph), provide all its categories and subcategories, and its brand.

B. Provide items from different subcategories that have the same brand.

C. Group products by brand and show the average price or rating for each brand.

D. Sort products in one category according to average brand price or rating.

E. Use an external service point, provide a description of 5 facts about the top brand from part D (e.g. https://query.wikidata.org/), e.g. location of headquarters. You may return images as one of your facts.

F. Recommend an item which is similar to the item using your linked RDF graph (i.e., shared properties and categories).

G. Write your own question about the webshop in plain English, then translate it to the corresponding SPARQL query, and run it on the graph. Provide a rationale for why this query would be valuable in a webshop setting, such as for semantic search or other applications.

In [8]:
'''
Query A
 For a given item (select a random item from your RDF Graph), provide all its categories and subcategories, and its brand.
'''
queryA = """
SELECT ?name ?subcategory ?category ?brand
WHERE {
  ?item schema:brand ?brand .                   #Get the brand
  ?item schema:name ?name.                      #Get item name
  ?item schema:item ?subcategory.            #Get the category.

OPTIONAL {?subcategory rdfs:subClassOf ?category }    # if there exist subcategory, also get it.
  }

ORDER BY RAND()
LIMIT 1
"""

resultA = graph.query(queryA)
for row in resultA:
    print('Name: ',row.name)
    if(row.category==None):
      print('Category: ',row.subcategory.replace("https://store.nintendo.nl/nl/","").replace("",""))
    if(row.category!=None):
      print('Category: ',row.category.replace("https://store.nintendo.nl/nl/","").replace("",""))
      print('Subcategory: ',row.subcategory.replace(row.category+"/","").replace("",""))
    print('Brand: ',row.brand)

Name:  Nintendo Switch – OLED-model The Legend of Zelda: Tears of the Kingdom Edition 
Category:  consoles
Subcategory:  https://store.nintendo.nl/nl/nintendo-switch-consoles
Brand:  Nintendo


In [9]:
'''
Query B
 Provide items from different subcategories that have the same brand
'''
queryB = """
SELECT ?name ?subcategory
WHERE {

  {
    #SAMPLE: We choose 1 item from each subcategory
    SELECT DISTINCT ?subcategory (SAMPLE(?item) AS ?sampleItem)
    WHERE {
    ?item schema:brand ?brand . #choose same brand
    ?item schema:name ?name .
    ?item schema:item ?subcategory .
    }
  GROUP BY ?subcategory
  }

  ?sampleItem schema:brand ?brand .
  ?sampleItem schema:name ?name .
  ?sampleItem schema:item ?subcategory .
  }
ORDER BY RAND()
"""

resultB = graph.query(queryB)
for row in resultB:
    print('Name: ',row.name)
    print('subCategory: ',row.subcategory.replace("https://store.nintendo.nl/nl/","").replace("",""))

Name:  Game Boy-posterset
subCategory:  home-and-gifts
Name:  Beenband
subCategory:  accessories
Name:  Nintendo Switch (neonblauw/neonrood) Animal Crossing: New Horizons-bundel
subCategory:  nintendo-switch-bundles
Name:  Animal Crossing: New Horizons
subCategory:  games
Name:  Kirby's Return to Dream Land Deluxe-muismat
subCategory:  franchises
Name:  Nintendo Switch met neonblauwe/neonrode Joy-Con-controllers
subCategory:  nintendo-switch-consoles
Name:  Kirby's Dream Buffet-sleutelhanger
subCategory:  apparel
Name:  Another Code: Recollection
subCategory:  new


In [10]:
'''
Query C
 Group products by brand and show the average price or rating for each brand.
'''
queryC = """
SELECT ?brand (AVG(?price) AS ?averagePrice)
WHERE {
  ?item schema:brand ?brand .
  ?item schema:price ?price .
}
GROUP BY ?brand
"""

resultC = graph.query(queryC)
for row in resultC:
    print('Brand: ',row.brand)
    print('Average Price: ',row.averagePrice)

Brand:  Nintendo
Average Price:  101.22698113207538


In [11]:
'''
Query D
 Sort products in one category according to average brand price or rating.
 Note: we only have one brand here, so we sort it according to its price
'''
queryD = """
SELECT ?item ?category ?price
WHERE {
  {
    #Here we choose a random category
    SELECT ?category
    WHERE {
      ?item schema:item ?category .
    }
    ORDER BY RAND()
    LIMIT 1
  }

  ?item schema:item ?category.    #Get all items within that category
  ?item schema:price ?price .

}
ORDER BY ?price   #sort by price
"""

resultD = graph.query(queryD)
for row in resultD:
    print('category: ',row.category.replace("https://store.nintendo.nl/nl/","").replace("",""))
    print('item: ',row.item)
    print('price: ',row.price)

category:  new
item:  https://store.nintendo.nl/nl/pikmin-4-theedoek-000000000010012315
price:  0.0
category:  new
item:  https://store.nintendo.nl/nl/https://www.nintendo.nl/Games/Nintendo-Switch-games/Metroid-Prime-Remastered-2341668.html
price:  39.99
category:  new
item:  https://store.nintendo.nl/nl/minecraft-legends-deluxe-edition-P00106
price:  49.99
category:  new
item:  https://store.nintendo.nl/nl/fae-farm-P00115
price:  59.99
category:  new
item:  https://store.nintendo.nl/nl/super-mario-rpg-70010000068684
price:  59.99
category:  new
item:  https://store.nintendo.nl/nl/pikmin-4-70010000005306
price:  59.99
category:  new
item:  https://store.nintendo.nl/nl/another-code-recollection-70010000072197
price:  59.99
category:  new
item:  https://store.nintendo.nl/nl/princess-peach-showtime-70010000005021
price:  59.99
category:  new
item:  https://store.nintendo.nl/nl/nintendo-switch-oled-model-the-legend-of-zelda-tears-of-the-kingdom-edition-the-legend-of-zelda-tears-of-the-king

In [12]:
'''
Query D
 Sort products in one category according to average brand price or rating.
'''
queryD = """
SELECT ?item ?category ?price
WHERE {
  {
    SELECT ?brand (AVG(?price) AS ?averagePrice)
    WHERE {
      ?item schema:brand ?brand .
      ?item schema:price ?price .
    }
  GROUP BY ?brand
  }

  {SELECT ?category #Here we choose a random category
    WHERE {
      ?item schema:item ?category .
    }
    ORDER BY RAND()
    LIMIT 1
  }

  ?item schema:item ?category.    
  ?item schema:price ?price .

}
ORDER BY ?averagePrice   #sort by average price
"""

resultD = graph.query(queryD)
for row in resultD:
    print('item: ',row.item)
    print('category', row.category.replace("https://store.nintendo.nl/nl/","").replace("",""))
    print('price: ',row.price)

item:  https://store.nintendo.nl/nl/fae-farm-P00115
category new
price:  59.99
item:  https://store.nintendo.nl/nl/nintendo-switch-oled-model-the-legend-of-zelda-tears-of-the-kingdom-edition-the-legend-of-zelda-tears-of-the-kingdom-bundel-B00396
category new
price:  429.99
item:  https://store.nintendo.nl/nl/super-mario-rpg-70010000068684
category new
price:  59.99
item:  https://store.nintendo.nl/nl/https://www.nintendo.nl/Games/Nintendo-Switch-games/Metroid-Prime-Remastered-2341668.html
category new
price:  39.99
item:  https://store.nintendo.nl/nl/minecraft-legends-deluxe-edition-P00106
category new
price:  49.99
item:  https://store.nintendo.nl/nl/pikmin-4-70010000005306
category new
price:  59.99
item:  https://store.nintendo.nl/nl/another-code-recollection-70010000072197
category new
price:  59.99
item:  https://store.nintendo.nl/nl/princess-peach-showtime-70010000005021
category new
price:  59.99
item:  https://store.nintendo.nl/nl/pikmin-4-theedoek-000000000010012315
category n

In [13]:
'''
Query E
E. Use an external service point, provide a description of 5 facts about the top brand from part D (e.g. https://query.wikidata.org/), e.g. location of headquarters. You may return images as one of your facts.
'''
import requests

url = 'https://query.wikidata.org/sparql'
queryE = '''
SELECT ?property ?propertyLabel (SAMPLE(?value) as ?sampleValue) (SAMPLE(?valueLabel) as ?sampleValueLabel)
WHERE {
  wd:Q8093 ?property ?value.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  FILTER (?property IN (wdt:P154, wdt:P159, wdt:P169,  wdt:P452,  wdt:P1448))
  
  #Q8093: Nintendo
  #154: logo image
  #159: headquarters location
  #452: industry
  #1448: official name
  #169: chief executive officer
}
GROUP BY ?property ?propertyLabel
'''
r = requests.get(url, params = {'format': 'json', 'query': queryE})
data = r.json()
resultE = []
for each in data['results']['bindings']:
    resultE.append(each['sampleValue']['value'])
resultE

['http://commons.wikimedia.org/wiki/Special:FilePath/Nintendo.svg',
 'http://www.wikidata.org/entity/Q34600',
 'http://www.wikidata.org/entity/Q52177144',
 'http://www.wikidata.org/entity/Q941594',
 'Nintendo Co., Ltd.']

In [14]:
'''
Query F
F. Recommend an item which is similar to the item using your linked RDF graph (i.e., shared properties and categories).
'''
queryF = """
SELECT DISTINCT ?name1 ?name2 ?price1 ?price2 ?category
WHERE {
    ?item1 schema:price ?price1.
    ?item2 schema:price ?price2.

    # recommand item with similar price:
    ?item1 schema:name ?name1.
    ?item2 schema:name ?name2.

    ?item1 schema:item ?category.
    ?item2 schema:item ?category.

    FILTER (ABS(?price1 - ?price2) <= 1)
}
ORDER BY RAND()
Limit 1
"""

resultF = graph.query(queryF)
for row in resultF:
    print('item: ',row.name1)
    print('price: ',row.price1)
    print('category: ',row.category.replace("https://store.nintendo.nl/nl/","").replace("",""))

    print('similar item: ',row.name2)  
    print('With price: ',row.price2)
    print('With category: ',row.category.replace("https://store.nintendo.nl/nl/","").replace("",""))

item:  Shin Megami Tensei V
price:  59.99
category:  games
similar item:  Super Mario Party™
With price:  59.99
With category:  games


In [15]:
'''
Query G
Write your own question about the webshop in plain English, then translate it to the corresponding SPARQL query, and run it on the graph. Provide a rationale for why this query would be valuable in a webshop setting, such as for semantic search or other applications.

Question: what are the price of Switch? list their prices from low to high.
'''

queryG = """
SELECT ?name  ?price
  WHERE {
    ?switch schema:item <https://store.nintendo.nl/nl/nintendo-switch-consoles>.
    ?switch schema:name ?name .
    ?switch schema:price ?price .
}
ORDER BY ?price   #sort by price
"""

resultG = graph.query(queryG)
for row in resultG:
       print('Name: ',row.name)
       print('Price: ',row.price)

Name:  Nintendo Switch met neonblauwe/neonrode Joy-Con-controllers
Price:  299.99
Name:  Nintendo Switch – OLED-Model Mario Edition (rood)
Price:  349.99
Name:  Nintendo Switch - OLED-model (neonblauw/neonrood)
Price:  349.99
Name:  Nintendo Switch - OLED-model (wit)
Price:  349.99
Name:  Nintendo Switch – OLED Model Pokémon Scarlet & Violet Edition
Price:  359.99
Name:  Nintendo Switch™ – OLED-Model Splatoon 3 Edition
Price:  359.99
Name:  Nintendo Switch – OLED-model The Legend of Zelda: Tears of the Kingdom Edition 
Price:  359.99


In [16]:
# Check the full graph: for checking only
query = """
SELECT * WHERE {
  ?s ?p ?o
}
"""
results = graph.query(query)
for row in results:
    print(row)

(rdflib.term.URIRef('https://store.nintendo.nl/nl/pokemon-legends-arceus-P00045'), rdflib.term.Literal('Vang, volg en onderzoek wilde Pokémon om de eerste Pokédex van de regio Sinnoh in Pokémon Legends: Arceus te maken en te voltooien._x000D_\r\n'), rdflib.term.URIRef('https://schema.org/description'))
(rdflib.term.URIRef('https://store.nintendo.nl/nl/animal-crossing-new-horizons-70010000027620'), rdflib.term.URIRef('https://schema.org/Product'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('https://store.nintendo.nl/nl/metroid-dread-70010000042925'), rdflib.term.Literal('70010000042925'), rdflib.term.URIRef('https://schema.org/sku'))
(rdflib.term.URIRef('https://store.nintendo.nl/nl/super-mario-rpg-70010000068684'), rdflib.term.Literal('59.99', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.URIRef('https://schema.org/price'))
(rdflib.term.URIRef('https://store.nintendo.nl/nl/nintendo-switch-oled-model-